In [1]:
from google.colab import drive
drive.mount('/content/drive')
!git config --global user.email "tzuhsin0502@gmail.com"
!git config --global user.name "110306058zihsin"
!git clone https://github.com/110306058zihsin/body-pose-forecasting.git
%cd body-pose-forecasting
!git checkout hsin
!git remote set-url origin git@github.com:110306058zihsin/body-pose-forecasting.git

Mounted at /content/drive


#Install library

In [12]:
!pip install git+https://github.com/nghorbani/human_body_prior
!pip install omegaconf
!pip install loguru
!pip install trimesh

  Cloning https://github.com/nghorbani/human_body_prior to /tmp/pip-req-build-6m5riqo8
  Running command git clone --filter=blob:none --quiet https://github.com/nghorbani/human_body_prior /tmp/pip-req-build-6m5riqo8
  Resolved https://github.com/nghorbani/human_body_prior to commit 78c86eae5ed518ae22bf197fd74211bbfa45551a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 874.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 829.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5

In [13]:
#filenames for loading VPoser VAE network, neutral SMPL body model, AMASS sample data

from os import path as osp

support_dir = '/content/drive/MyDrive/VPoserModelFiles/'

expr_dir = osp.join(support_dir,'vposer_v2_05/') #'TRAINED_MODEL_DIRECTORY'
bm_fname =  osp.join(support_dir,'smplx_neutral_model.npz')    #'PATH_TO_SMPLX_model.npz'  neutral smpl body model
sample_amass_fname = osp.join(support_dir, 'amass_sample.npz')  # a sample npz file from AMASS


print(expr_dir)
print(bm_fname)
print(sample_amass_fname)

/content/drive/MyDrive/VPoserModelFiles/vposer_v2_05/
/content/drive/MyDrive/VPoserModelFiles/smplx_neutral_model.npz
/content/drive/MyDrive/VPoserModelFiles/amass_sample.npz


In [14]:
#Loading SMPLx Body Model
import torch
import numpy as np

# Choose the device to run the body model on, cuda or cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('device is', device)

from human_body_prior.body_model.body_model import BodyModel
bm = BodyModel(bm_fname=bm_fname).to(device)

device is cpu


In [ ]:
#Loading VPoser VAE Body Pose Prior
from human_body_prior.tools.model_loader import load_model
from human_body_prior.models.vposer_model import VPoser

vp, ps = load_model(expr_dir, model_code=VPoser,
                              remove_words_in_model_weights='vp_model.',
                              disable_grad=True,
                              comp_device=device)
vp = vp.to(device)

2026-02-19 16:13:27.612 | INFO     | human_body_prior.tools.model_loader:load_model:77 - No GPU detected. Loading on CPU!
/usr/local/lib/python3.12/dist-packages/human_body_prior/tools/model_loader.py:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please o

In [ ]:
# Prepare the body poses from amass sample file
#  indexing [3:66] removes global rotation, hands/fingers, and anything else other than 21 major body joints
amass_body_pose = np.load(sample_amass_fname)['poses'][:, 3:66]
amass_body_pose = torch.from_numpy(amass_body_pose).type(torch.float).to(device)
print('amass_body_pose.shape', amass_body_pose.shape)

In [ ]:
# run the encoder on all frames
amass_body_poZ = vp.encode(amass_body_pose).mean
print('amass_body_poZ.shape', amass_body_poZ.shape)

In [ ]:
# run the decoder on all frames
amass_body_pose_rec = vp.decode(amass_body_poZ)['pose_body'].contiguous().view(-1, 63)
print('amass_body_pose_rec.shape', amass_body_pose_rec.shape)